In [1]:
import math
from random import sample
from torch import empty

from generate_data import generate_data
from Modules import Sequential, Linear, ReLU, Tanh, Sigmoid, LossMSE

In [2]:
N = 1000

train_input, train_target, test_input, test_target = generate_data(1000, True)

print(train_input.shape)
print(train_input.std())
print(train_input.mean())

torch.Size([1000, 2])
tensor(1.0000)
tensor(-1.1122e-07)


In [3]:
l1 = Linear(4,6)
r1 = ReLU()
l2 = Linear(6, 8)
t2 = Tanh()

S = Sequential(l1, r1, l2, t2)
print(S.forward(empty(4).normal_()))
print(S.backward(empty(8).normal_()))
print(S.param())
S.name

tensor([ 1., -1.,  1.,  1.,  1.,  1.,  1., -1.])
tensor([0., 0., 0., 0.])
ReLU has no parameters
Tanh has no parameters
[[(tensor([[-0.4741, -1.0968,  0.0198,  0.7538],
        [-0.6199,  1.2094, -1.2085, -0.4808],
        [-0.2418, -0.7311,  0.5526, -0.7504],
        [ 0.1559,  2.0285,  0.8230,  0.2036],
        [ 0.4077, -0.6128, -1.0538, -0.8381],
        [-0.9060,  0.2269,  0.3754,  0.7649]]), tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])), (tensor([6.6764e-07, 4.9153e-14, 1.3563e-19, 1.8578e-01, 3.9178e-02, 4.7429e+30]), tensor([0., 0., 0., 0., 0., 0.]))], None, [(tensor([[ 0.4660,  0.9695,  0.5439, -0.2080,  2.0194,  1.6116],
        [ 0.0056,  0.4408, -0.3127, -1.2531, -1.1860, -1.7312],
        [-0.2487, -0.0309,  0.4719, -1.2130,  0.3731,  0.5068],
        [ 0.9429,  1.0302, -0.9563,  0.3355, -0.1102,  2.1731],
        [-0.7564, -0.5229,  0.5746, -1.4651, -0.0496,  0.

'Linear(4x6) => ReLU => Linear(6x8) => Tanh'

In [4]:
def one_hot_encode(target):
    ohe_target = empty(target.shape[0], 2)
    ohe_target[:, 0] = train_target == 0
    ohe_target[:, 1] = train_target == 1
    return ohe_target

In [29]:
def train_model_SGD(model, train_input, train_target, nb_epoch=35, mini_batch_size=1):
    nb_train_samples = train_input.shape[0]
    ohe_train_target = one_hot_encode(train_target)
    mse = LossMSE()
    eta = 1e-1 / nb_train_samples
    nb_batch_per_epoch = nb_train_samples//mini_batch_size
    losses = empty(nb_epoch, nb_batch_per_epoch)
    for epoch in range(nb_epoch):
        for batch in range(nb_batch_per_epoch):
            loss = 0
            indices = sample(range(nb_train_samples), mini_batch_size)
            for i in indices:
                # Run forward pass
                output = model.forward(train_input[i])
                # Compute loss
                loss += mse.compute_loss(output, ohe_train_target[i])
                
                # Run back propagation
                model.backward(mse.gradient())
            
            loss /= nb_batch_per_epoch
            losses[epoch][batch] = loss
            model.update_param(eta)
            model.zero_grad()
        
        mean_loss = losses.mean(1)[epoch]
        print(f"Epoch: {epoch}, Mean loss: {mean_loss}")

In [30]:
l1 = Linear(2,25)
a1 = ReLU()
l2 = Linear(25, 25)
a2 = Tanh()
l3 = Linear(25, 25)
a3 = Sigmoid()
l4 = Linear(25, 2)

model = Sequential(l1, a1, l2, a2, l3, a3, l4)

In [31]:
train_model_SGD(model, train_input, train_target)

Epoch: 0, Mean loss: 0.005780851934105158
Epoch: 1, Mean loss: 0.0011925232829526067
Epoch: 2, Mean loss: 0.0007115315529517829
Epoch: 3, Mean loss: 0.0005163089954294264
Epoch: 4, Mean loss: 0.0003799007972702384
Epoch: 5, Mean loss: 0.0003064349584747106
Epoch: 6, Mean loss: 0.00029846528195776045
Epoch: 7, Mean loss: 0.00022276448726188391
Epoch: 8, Mean loss: 0.00022204728156793863
Epoch: 9, Mean loss: 0.00020371220307424664
Epoch: 10, Mean loss: 0.00016944350500125438
Epoch: 11, Mean loss: 0.00015764450654387474
Epoch: 12, Mean loss: 0.0001564516860526055
Epoch: 13, Mean loss: 0.00014290765102487057
Epoch: 14, Mean loss: 0.00014341293717734516
Epoch: 15, Mean loss: 0.00013695152301806957
Epoch: 16, Mean loss: 0.00013215671060606837
Epoch: 17, Mean loss: 0.00011268461821600795
Epoch: 18, Mean loss: 0.00011057652591262013
Epoch: 19, Mean loss: 0.0001138973530032672
Epoch: 20, Mean loss: 0.00010396025027148426
Epoch: 21, Mean loss: 0.00010465495142852888
Epoch: 22, Mean loss: 0.00011

In [32]:
def accuracy(model, input, target):
    output = empty(input.shape[0], 2)
    for n in range(output.shape[0]):
        output[n] = model.forward(train_input[n])
    return (output.argmax(1) == target).sum() / float(output.shape[0])

In [33]:
accuracy(model, train_input, train_target)

tensor(0.9760)

In [34]:
accuracy(model, test_input, test_target)

tensor(0.5590)